In [ ]:
# nb46_narrative_style_control.ipynb
# Goals: Narrative style control for text adventure games
# - Dynamic style switching (genre/tone/length)
# - Style Dictionary integration
# - Consistency validation

# Cell1:  Shared Cache Bootstrap
import os, pathlib, torch
import sys
from datetime import datetime

# Shared cache configuration (複製到每本 notebook)
AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "../ai_warehouse/cache")

for k, v in {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)
print("[Cache]", AI_CACHE_ROOT, "| GPU:", torch.cuda.is_available())

In [ ]:
# Cell 2: Dependencies and Setup
import json
import yaml
import re
from typing import Dict, List, Optional, Any
from dataclasses import dataclass, asdict
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch.nn.functional as F


# Simple LLM adapter for demonstration
class LLMAdapter:
    def __init__(self, model_id="Qwen/Qwen2.5-7B-Instruct"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id, device_map="auto", torch_dtype="auto"
        )
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def generate(self, prompt: str, max_length=256, temperature=0.7):
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_length,
                temperature=temperature,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
            )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)


# Initialize LLM
llm = LLMAdapter()
print(f"✓ LLM loaded: {llm.model.config.name_or_path}")

In [ ]:
# Cell 3: Style Dictionary Core System
@dataclass
class StyleConfig:
    """Narrative style configuration"""

    name: str
    genre: str  # wuxia, scifi, fantasy, modern
    tone: str  # formal, casual, dramatic, mysterious
    length: str  # brief, medium, detailed, verbose
    perspective: str  # first, second, third
    tense: str  # present, past
    vocabulary_level: str  # simple, standard, advanced, archaic
    description_density: str  # sparse, balanced, rich, overwhelming


class StyleDictionary:
    """Style Dictionary management system"""

    def __init__(self):
        self.styles = {}
        self.active_style = None
        self._load_default_styles()

    def _load_default_styles(self):
        """Load predefined style templates"""
        default_styles = {
            "wuxia_classical": StyleConfig(
                name="古典武俠",
                genre="wuxia",
                tone="formal",
                length="detailed",
                perspective="third",
                tense="past",
                vocabulary_level="archaic",
                description_density="rich",
            ),
            "scifi_modern": StyleConfig(
                name="現代科幻",
                genre="scifi",
                tone="casual",
                length="medium",
                perspective="second",
                tense="present",
                vocabulary_level="standard",
                description_density="balanced",
            ),
            "fantasy_epic": StyleConfig(
                name="史詩奇幻",
                genre="fantasy",
                tone="dramatic",
                length="verbose",
                perspective="third",
                tense="past",
                vocabulary_level="advanced",
                description_density="overwhelming",
            ),
            "mystery_noir": StyleConfig(
                name="懸疑黑色",
                genre="mystery",
                tone="mysterious",
                length="brief",
                perspective="first",
                tense="past",
                vocabulary_level="standard",
                description_density="sparse",
            ),
        }

        for key, style in default_styles.items():
            self.styles[key] = style

    def set_active_style(self, style_key: str):
        """Set active narrative style"""
        if style_key in self.styles:
            self.active_style = self.styles[style_key]
            return True
        return False

    def get_style_prompt(self) -> str:
        """Generate style-specific prompt instructions"""
        if not self.active_style:
            return ""

        style = self.active_style

        # Genre-specific vocabulary and themes
        genre_rules = {
            "wuxia": "使用武俠小說風格，包含功夫、江湖、俠義等元素。多用古典詞彙如「閣下」「在下」「施主」。",
            "scifi": "使用科幻風格，包含科技、太空、未來等元素。術語應現代化且具科技感。",
            "fantasy": "使用奇幻風格，包含魔法、龍、精靈等元素。創造富有想像力的描述。",
            "mystery": "使用懸疑風格，營造神秘氣氛。描述應簡潔有力，留有懸念。",
        }

        # Tone guidelines
        tone_rules = {
            "formal": "使用正式、莊重的語調。避免口語化表達。",
            "casual": "使用輕鬆、自然的語調。可適度使用口語。",
            "dramatic": "使用戲劇化、情感豐富的語調。強調衝突與張力。",
            "mysterious": "使用神秘、模糊的語調。多用暗示而非直述。",
        }

        # Length guidelines
        length_rules = {
            "brief": "保持簡潔，每段落50-100字。突出關鍵資訊。",
            "medium": "適中篇幅，每段落100-200字。平衡描述與行動。",
            "detailed": "詳細描述，每段落200-300字。豐富場景與人物刻劃。",
            "verbose": "極其詳細，每段落300字以上。深入探索各個細節。",
        }

        # Perspective and tense
        perspective_rules = {
            "first": "使用第一人稱（我）視角敘述。",
            "second": "使用第二人稱（你）視角敘述。",
            "third": "使用第三人稱視角敘述。",
        }

        tense_rules = {
            "present": "使用現在式敘述，增強臨場感。",
            "past": "使用過去式敘述，如傳統故事。",
        }

        prompt_parts = [
            f"風格指南：{style.name}",
            f"類型：{genre_rules.get(style.genre, '')}",
            f"語調：{tone_rules.get(style.tone, '')}",
            f"篇幅：{length_rules.get(style.length, '')}",
            f"視角：{perspective_rules.get(style.perspective, '')}",
            f"時態：{tense_rules.get(style.tense, '')}",
        ]

        return "\n".join(prompt_parts)

    def list_styles(self) -> List[str]:
        """List available styles"""
        return [(k, v.name) for k, v in self.styles.items()]


# Initialize style system
style_dict = StyleDictionary()
print("✓ Style Dictionary loaded with styles:")
for key, name in style_dict.list_styles():
    print(f"  - {key}: {name}")

In [ ]:
# Cell 4: Narrative Style Templates
class NarrativeGenerator:
    """Generate narrative content with style control"""

    def __init__(self, llm_adapter: LLMAdapter, style_dict: StyleDictionary):
        self.llm = llm_adapter
        self.style_dict = style_dict

    def generate_scene_description(self, scene_data: Dict[str, Any]) -> str:
        """Generate styled scene description"""
        style_prompt = self.style_dict.get_style_prompt()

        base_prompt = f"""你是一位專業的文字冒險遊戲敘事者。請根據以下風格指南和場景資訊，生成引人入勝的場景描述。

{style_prompt}

場景資訊：
- 地點：{scene_data.get('location', '未知地點')}
- 時間：{scene_data.get('time', '某個時刻')}
- 天氣：{scene_data.get('weather', '普通天氣')}
- 氣氛：{scene_data.get('atmosphere', '中性')}
- 重要物件：{scene_data.get('objects', [])}
- 在場角色：{scene_data.get('characters', [])}

請生成符合指定風格的場景描述："""

        response = self.llm.generate(base_prompt, max_length=300, temperature=0.8)
        # Extract only the generated description (remove prompt)
        lines = response.split("\n")
        start_idx = -1
        for i, line in enumerate(lines):
            if "請生成符合指定風格的場景描述：" in line:
                start_idx = i + 1
                break

        if start_idx > 0:
            return "\n".join(lines[start_idx:]).strip()
        return response.split("\n")[-1].strip()

    def generate_character_description(self, character_data: Dict[str, Any]) -> str:
        """Generate styled character description"""
        style_prompt = self.style_dict.get_style_prompt()

        base_prompt = f"""請根據風格指南描述以下角色：

{style_prompt}

角色資訊：
- 姓名：{character_data.get('name', '神秘人')}
- 職業：{character_data.get('profession', '未知')}
- 外貌特徵：{character_data.get('appearance', '普通外貌')}
- 性格：{character_data.get('personality', '中性性格')}
- 當前狀態：{character_data.get('state', '正常')}

請生成角色描述："""

        response = self.llm.generate(base_prompt, max_length=200, temperature=0.7)
        lines = response.split("\n")
        for i, line in enumerate(lines):
            if "請生成角色描述：" in line:
                return "\n".join(lines[i + 1 :]).strip()
        return response.split("\n")[-1].strip()

    def generate_action_result(self, action_data: Dict[str, Any]) -> str:
        """Generate styled action result description"""
        style_prompt = self.style_dict.get_style_prompt()

        base_prompt = f"""請根據風格指南描述行動結果：

{style_prompt}

行動資訊：
- 玩家行動：{action_data.get('action', '未知行動')}
- 結果類型：{action_data.get('result_type', 'success')}  # success/failure/partial
- 影響：{action_data.get('impact', '輕微影響')}
- 後續線索：{action_data.get('clues', [])}

請生成行動結果描述："""

        response = self.llm.generate(base_prompt, max_length=250, temperature=0.75)
        lines = response.split("\n")
        for i, line in enumerate(lines):
            if "請生成行動結果描述：" in line:
                return "\n".join(lines[i + 1 :]).strip()
        return response.split("\n")[-1].strip()


# Initialize narrative generator
narrator = NarrativeGenerator(llm, style_dict)
print("✓ Narrative Generator initialized")

In [ ]:
# Cell 5: Dynamic Style Injection Demo
def demonstrate_style_switching():
    """Demonstrate same scene with different styles"""

    # Common scene data
    scene_data = {
        "location": "古老的圖書館",
        "time": "深夜",
        "weather": "雷雨交加",
        "atmosphere": "神秘詭異",
        "objects": ["古籍", "燭台", "神秘符文"],
        "characters": ["蒙面守護者"],
    }

    results = {}

    print("=== 同場景不同風格演示 ===\n")

    for style_key, style_name in style_dict.list_styles():
        print(f"🎭 {style_name} ({style_key})")
        print("-" * 40)

        # Set style and generate
        style_dict.set_active_style(style_key)
        description = narrator.generate_scene_description(scene_data)

        print(f"風格設定：")
        style_config = style_dict.active_style
        print(f"  類型：{style_config.genre} | 語調：{style_config.tone}")
        print(f"  篇幅：{style_config.length} | 視角：{style_config.perspective}")
        print(f"\n生成描述：")
        print(f"「{description}」")
        print("\n" + "=" * 60 + "\n")

        results[style_key] = {
            "style_name": style_name,
            "description": description,
            "word_count": len(description),
        }

    return results


# Run style switching demo
style_demo_results = demonstrate_style_switching()

In [ ]:
# Cell 6: Style Consistency Validation
class StyleValidator:
    """Validate narrative consistency with selected style"""

    def __init__(self):
        self.style_keywords = {
            "wuxia": ["武功", "江湖", "俠客", "劍氣", "内功", "輕功", "閣下", "在下"],
            "scifi": ["科技", "太空", "未來", "機器人", "星球", "能量", "系統", "數據"],
            "fantasy": ["魔法", "精靈", "龍", "法術", "魔力", "咒語", "魔導師", "元素"],
            "mystery": ["神秘", "線索", "謎團", "隱藏", "秘密", "暗示", "疑點", "真相"],
        }

        self.tone_indicators = {
            "formal": ["閣下", "敝人", "謹慎", "恭敬", "莊重"],
            "casual": ["你", "我", "挺", "很", "超"],
            "dramatic": ["！", "驚", "震撼", "澎湃", "激烈", "壯烈"],
            "mysterious": ["似乎", "彷彿", "隱約", "模糊", "若隱若現", "神秘"],
        }

    def validate_genre_consistency(self, text: str, expected_genre: str) -> float:
        """Check if text matches expected genre (0-1 score)"""
        if expected_genre not in self.style_keywords:
            return 0.5  # Unknown genre

        keywords = self.style_keywords[expected_genre]
        matches = sum(1 for keyword in keywords if keyword in text)
        return min(matches / 3, 1.0)  # Normalize to 0-1, cap at 1.0

    def validate_tone_consistency(self, text: str, expected_tone: str) -> float:
        """Check if text matches expected tone (0-1 score)"""
        if expected_tone not in self.tone_indicators:
            return 0.5

        indicators = self.tone_indicators[expected_tone]
        matches = sum(1 for indicator in indicators if indicator in text)
        return min(matches / 2, 1.0)

    def validate_length_consistency(self, text: str, expected_length: str) -> float:
        """Check if text length matches expectation (0-1 score)"""
        char_count = len(text)

        length_ranges = {
            "brief": (50, 150),
            "medium": (100, 250),
            "detailed": (200, 400),
            "verbose": (300, 600),
        }

        if expected_length not in length_ranges:
            return 0.5

        min_len, max_len = length_ranges[expected_length]

        if min_len <= char_count <= max_len:
            return 1.0
        elif char_count < min_len:
            return char_count / min_len
        else:  # char_count > max_len
            return max_len / char_count

    def comprehensive_validation(
        self, text: str, style_config: StyleConfig
    ) -> Dict[str, float]:
        """Comprehensive style validation"""
        return {
            "genre_consistency": self.validate_genre_consistency(
                text, style_config.genre
            ),
            "tone_consistency": self.validate_tone_consistency(text, style_config.tone),
            "length_consistency": self.validate_length_consistency(
                text, style_config.length
            ),
            "overall_score": 0.0,  # Will be calculated
        }


# Initialize validator
validator = StyleValidator()

# Validate demo results
print("=== 風格一致性驗證 ===\n")
for style_key, result in style_demo_results.items():
    style_config = style_dict.styles[style_key]
    validation = validator.comprehensive_validation(result["description"], style_config)

    # Calculate overall score
    validation["overall_score"] = (
        validation["genre_consistency"] * 0.4
        + validation["tone_consistency"] * 0.3
        + validation["length_consistency"] * 0.3
    )

    print(f"🔍 {result['style_name']} 驗證結果：")
    print(f"  類型一致性：{validation['genre_consistency']:.2f}")
    print(f"  語調一致性：{validation['tone_consistency']:.2f}")
    print(f"  篇幅一致性：{validation['length_consistency']:.2f}")
    print(f"  總體評分：{validation['overall_score']:.2f}")
    print(f"  字數：{result['word_count']}\n")

print("✓ Style validation completed")

In [ ]:
# Cell 7: Interactive Style Control System
class GameStyleController:
    """Game-integrated style control system"""

    def __init__(self, narrator: NarrativeGenerator, validator: StyleValidator):
        self.narrator = narrator
        self.validator = validator
        self.style_history = []
        self.consistency_scores = []

    def switch_style(self, new_style_key: str) -> bool:
        """Switch narrative style with validation"""
        success = self.narrator.style_dict.set_active_style(new_style_key)
        if success:
            self.style_history.append(
                {
                    "style_key": new_style_key,
                    "timestamp": "demo_time",
                    "style_name": self.narrator.style_dict.styles[new_style_key].name,
                }
            )
        return success

    def generate_with_validation(
        self, content_type: str, data: Dict[str, Any]
    ) -> Dict[str, Any]:
        """Generate content with automatic validation"""
        # Generate content based on type
        if content_type == "scene":
            text = self.narrator.generate_scene_description(data)
        elif content_type == "character":
            text = self.narrator.generate_character_description(data)
        elif content_type == "action":
            text = self.narrator.generate_action_result(data)
        else:
            return {"error": f"Unknown content type: {content_type}"}

        # Validate consistency
        current_style = self.narrator.style_dict.active_style
        validation = self.validator.comprehensive_validation(text, current_style)

        # Record consistency score
        self.consistency_scores.append(validation["overall_score"])

        return {
            "text": text,
            "validation": validation,
            "style_used": current_style.name,
            "word_count": len(text),
        }

    def get_style_report(self) -> Dict[str, Any]:
        """Generate style usage and consistency report"""
        if not self.consistency_scores:
            return {"error": "No generation history available"}

        return {
            "total_generations": len(self.consistency_scores),
            "average_consistency": sum(self.consistency_scores)
            / len(self.consistency_scores),
            "style_switches": len(self.style_history),
            "current_style": (
                self.narrator.style_dict.active_style.name
                if self.narrator.style_dict.active_style
                else None
            ),
            "consistency_trend": (
                self.consistency_scores[-5:]
                if len(self.consistency_scores) >= 5
                else self.consistency_scores
            ),
        }


# Initialize style controller
game_style = GameStyleController(narrator, validator)
print("✓ Game Style Controller initialized")

In [ ]:
# Cell 8: Comprehensive Style Control Demo
def comprehensive_style_demo():
    """Complete demonstration of style control system"""

    print("=== 完整風格控制系統演示 ===\n")

    # Demo scenario data
    scenarios = [
        {
            "type": "scene",
            "data": {
                "location": "荒廢的神廟",
                "time": "黃昏",
                "weather": "陰雲密佈",
                "atmosphere": "莊嚴肅殺",
                "objects": ["破碎石像", "祭壇", "古老壁畫"],
                "characters": ["神秘祭司"],
            },
        },
        {
            "type": "character",
            "data": {
                "name": "影刃",
                "profession": "刺客",
                "appearance": "黑衣蒙面，身形矯健",
                "personality": "冷酷無情，技藝高超",
                "state": "潛伏待機",
            },
        },
        {
            "type": "action",
            "data": {
                "action": "偷偷潛入寶庫",
                "result_type": "partial",
                "impact": "觸發了警報機關",
                "clues": ["守衛腳步聲", "紅光閃爍", "石門緩緩關閉"],
            },
        },
    ]

    # Test different styles on same scenarios
    test_styles = ["wuxia_classical", "scifi_modern", "fantasy_epic"]

    for style_key in test_styles:
        print(f"🎭 測試風格：{style_dict.styles[style_key].name}")
        print("=" * 50)

        # Switch style
        game_style.switch_style(style_key)

        # Generate content for each scenario type
        for i, scenario in enumerate(scenarios, 1):
            print(f"\n📝 場景 {i} ({scenario['type']})：")
            result = game_style.generate_with_validation(
                scenario["type"], scenario["data"]
            )

            if "error" not in result:
                print(f"生成內容：「{result['text']}」")
                print(f"一致性評分：{result['validation']['overall_score']:.2f}")
                print(f"字數：{result['word_count']}")
            else:
                print(f"生成失敗：{result['error']}")

        print("\n" + "-" * 50 + "\n")

    # Generate final report
    report = game_style.get_style_report()
    print("📊 風格控制報告：")
    print(f"  總生成次數：{report.get('total_generations', 0)}")
    print(f"  平均一致性：{report.get('average_consistency', 0):.2f}")
    print(f"  風格切換次數：{report.get('style_switches', 0)}")
    print(f"  當前風格：{report.get('current_style', 'None')}")

    return report


# Run comprehensive demo
demo_report = comprehensive_style_demo()

In [ ]:
# Cell 9: Smoke Test
def smoke_test_style_control():
    """Quick smoke test for style control system"""
    print("🧪 風格控制系統煙霧測試")
    print("=" * 40)

    try:
        # Test 1: Style switching
        print("1. 測試風格切換...")
        success = game_style.switch_style("wuxia_classical")
        assert success, "風格切換失敗"
        print("   ✓ 風格切換成功")

        # Test 2: Content generation
        print("2. 測試內容生成...")
        test_scene = {
            "location": "竹林",
            "time": "月夜",
            "weather": "清風徐來",
            "atmosphere": "寧靜致遠",
        }
        result = game_style.generate_with_validation("scene", test_scene)
        assert "text" in result, "內容生成失敗"
        assert len(result["text"]) > 10, "生成內容過短"
        print(f"   ✓ 生成內容：{result['text'][:30]}...")

        # Test 3: Validation
        print("3. 測試風格驗證...")
        validation = result["validation"]
        assert "overall_score" in validation, "驗證失敗"
        assert 0 <= validation["overall_score"] <= 1, "驗證分數範圍錯誤"
        print(f"   ✓ 驗證分數：{validation['overall_score']:.2f}")

        # Test 4: Style listing
        print("4. 測試風格列表...")
        styles = style_dict.list_styles()
        assert len(styles) >= 4, "風格數量不足"
        print(f"   ✓ 可用風格：{len(styles)} 種")

        print("\n🎉 所有測試通過！風格控制系統運作正常。")
        return True

    except Exception as e:
        print(f"\n❌ 測試失敗：{str(e)}")
        return False


# Run smoke test
smoke_passed = smoke_test_style_control()

# Cell 10: Summary and Extensions
print("\n" + "=" * 60)
print("📋 NB46 總結：敘事風格控制系統")
print("=" * 60)

print(
    """
✅ 完成功能：
1. Style Dictionary 核心系統 - 支援多種風格配置
2. 動態風格切換機制 - 即時調整敘事風格
3. 風格模板庫 - 武俠/科幻/奇幻/懸疑風格
4. 一致性驗證系統 - 自動檢測風格符合度
5. 遊戲整合控制器 - 完整的風格管理API

🎯 核心概念：
- 風格配置結構化：類型/語調/篇幅/視角等維度
- 提示詞動態注入：根據風格自動調整生成指導
- 一致性量化評估：多維度評分機制
- 歷史追蹤記錄：風格使用與效果分析

⚠️ 常見陷阱：
- 風格過度切換：頻繁改變會造成敘事不連貫
- 驗證過於嚴格：可能限制創意表達的多樣性
- 模板僵化：避免風格指南過於死板，需保留靈活性
- 語言模型依賴：不同模型對風格指令的理解差異

🚀 擴展建議：
- 玩家偏好學習：根據選擇歷史推薦合適風格
- 情境自動切換：根據劇情發展自動調整風格
- 混合風格支援：允許多種風格元素混合使用
- 國際化支援：擴展到其他語言的風格控制
- 語音風格：整合TTS的語調與節奏控制

💡 實際應用：
- 文字冒險遊戲的沉浸感提升
- 小說創作工具的風格一致性
- 教育內容的個性化呈現
- 聊天機器人的人格塑造
"""
)

In [ ]:
# 快速驗證腳本
def quick_validation():
    # 1. 風格切換測試
    assert style_dict.set_active_style("wuxia_classical"), "武俠風格設定失敗"

    # 2. 內容生成測試
    test_data = {"location": "客棧", "time": "深夜"}
    text = narrator.generate_scene_description(test_data)
    assert len(text) > 20, "生成內容太短"

    # 3. 驗證功能測試
    validation = validator.comprehensive_validation(text, style_dict.active_style)
    assert 0 <= validation["overall_score"] <= 1, "驗證分數異常"

    print("✅ 核心功能驗證通過")


quick_validation()